In [6]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')

driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

C:\Users\Chloe\AppData\Local\Temp\ipykernel_22152\3096828039.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)


In [22]:
# creates a datafram with all of the needed columns including "title", "author", etc.
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'director': pd.Series(dtype='str'),
                   'release_year': pd.Series(dtype='str'),
                   'length': pd.Series(dtype='str'),
                   'genre': pd.Series(dtype='str'),
                   'imdb_rating': pd.Series(dtype='str'),
                   'gross': pd.Series(dtype='str')              
})

print(df)

#specifies the website to scrape and use selenium
driver.get("https://www.imdb.com/search/title/?title_type=feature&sort=alpha,asc")


i = 0
while True:
    try:
        # waits until the head-block class is displayed
        WebDriverWait(driver, 1).until(
            lambda s: s.find_element(By.CLASS_NAME, 'article').is_displayed()
        )
    except TimeoutException:
        break

    url = driver.current_url
    response = requests.get(url)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    
    time.sleep(1)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("div", class_="lister-item-content")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("h3", class_="lister-item-header").find("a").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_director = item_info.find("p", class_="").find("a").get_text(strip=True)
        except:
            movie_director = pd.NA
        try:
            movie_release = item_info.find("h3", class_="lister-item-header").find("span", class_="lister-item-year text-muted unbold").get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_length = item_info.find("p", class_="text-muted ").find("span", class_="runtime").get_text(strip=True)
        except:
            movie_length = pd.NA
        
        #since these values are sometimes non-existent, I use a try-except block
        #to set the value to NaN if there is an error (meaning the value doesn't exist)

        
        #adds each of these values as a new row to the dataframe
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'director': movie_director,
                       'release_year': movie_release,
                       'length': movie_length,
                                      
                      }).to_frame().T], ignore_index=True)

    #makes sure the next button exists before trying to click it
    try:
        load_more = driver.find_element(By.LINK_TEXT,'Next »')
        if load_more: driver.execute_script("arguments[0].click();", load_more)
    except: pass
                       

    #after each page it increments i by 1, once i is greater than 33 we have reached
    #the last page so it breaks out of the while statement
    i += 1
    if i > 2:
        break

#saves the dataframe as a csv file
df.to_csv('imdb_movies.csv')
df

Empty DataFrame
Columns: [title, director, release_year, length, genre, imdb_rating, gross]
Index: []


,title,director,release_year,length,genre,imdb_rating,gross
0,"""Giliap""",Roy Andersson,(1975),<NA>,NaN,NaN,NaN
1,"""Swing it"" magistern",Schamyl Bauman,(1940),<NA>,NaN,NaN,NaN
2,The Wind Bloweth Where It Listeth,Kim Ekberg,(2017),<NA>,NaN,NaN,NaN
3,# (Hashtag),<NA>,,<NA>,NaN,NaN,NaN
4,#1 Mamarazzi,Kevin Alan Kent,,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...
145,#babynymph,Aldo Pedrosa,(2016),<NA>,NaN,NaN,NaN
146,#blessed,Rich Costales,(2015),<NA>,NaN,NaN,NaN
147,#chadgetstheaxe,Travis Bible,(2022),<NA>,NaN,NaN,NaN
148,#dogpoopgirl,Andrei Hutuleac,(2021),<NA>,NaN,NaN,NaN
